## Connecting to HPDS resources 101

### 1. Manualing setting up a security token

To connect to a network, one need two informations: 
- URL of this network. This notebook example is using COPDGene Dev environment, which URL is https://copdgene-dev.hms.harvard.edu/psamaui/login/?redirection_url=/picsureui/
- An authorized individual user token to gain access to the resources of the network through the API.

We will hence need to get a token and feed it to the API. First, if it does not already exist, we will create a blank text file that will be used to store the token right after.

In [3]:
%%bash
[ ! -d "./tokens" ] && mkdir "./tokens" && echo "dir created"
[ ! -f "./tokens/copd.txt" ] && touch "./tokens/copd.txt" && echo "file created"

dir created
file created


To actually get the token, process as below:

1. In a web browser, open the COPDGene Dev User Interface: https://copdgene-dev.hms.harvard.edu/psamaui/login/?redirection_url=/picsureui/, and choose one of the available authentication methods to enter it.
2. In the user-interface click on USER PROFILE
3. On the new window, click again on USER PROFILE
4. On the pop-up window, click on REFRESH and then COPY
5. Back into your Jupyter environment, paste it into the newly created text file (`./tokens/copd.txt`).

![Getting authorization token](img/get_token_screen.png)

**Token is strictly personal**, be careful not to share it with anyone (thus `./tokens` directory is explicitely excluded in the `.gitignore` file).

## Accessing an HPDS-hosted database using Python PIC-SURE API

In [6]:
from python_lib.HPDS_connection_manager import tokenManager

We then use the user-defined `tokenManager` class to import the token. This class basically ensure that the token file specified exist and is not empty.

In [5]:
token = tokenManager(token_file).get_token()

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'tokenManager' is not defined


### 2. Accessing the databases

Gaining access to any data source using the API require two steps: 
* Connecting to a PIC-SURE network
* Connecting to an HPDS-hosted resource. Indeed, a PIC-sure network can host several different resources. However, the only resources we will be using hereafter is the COPDGene HPDS-hosted database instance.

In [ ]:
!pip install -r requirements.txt

In [ ]:
import PicSureHpdsLib
import PicSureClient

We will use the `PicSureClient` library to create the connection to a PICSure network, and then the `PicSureHpdsLib` library to handle data extraction from a HPDS-hosted database. 

#### Connecting to a PIC-SURE network

In [ ]:
PICSURE_network_URL = "https://copdgene-dev.hms.harvard.edu/picsure/"
COPDGene_resource = "b6ef7b1a-56f6-11e9-8958-0242c0a83007"

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, token, allowSelfSignedSSL=True)
#connection.list()

#### Connecting to a specific resource within the Network

Here we will connect to the *COPDGene* resource

In [ ]:
COPDGene_resource = "b6ef7b1a-56f6-11e9-8958-0242c0a83007"
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(COPDGene_resource)

Finally, we hence created an object called `resource`, which is an instance of the `PicSureHpdsLib.Adapter()` class. It is connected to the specific resources we indicated, namely COPDGene hosted database in our case. 

**This `resource` object is actually the only one we will need to proceed with any specific analysis thereafter**.

## Accessing an HPDS-hosted database using R PIC-SURE API

In [ ]:
devtools::install_github("hms-dbmi/pic-sure-r-client")
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds")

#### Loading user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connection to COPDgenes database

In [ ]:
credentials_dic = jsonlite::read_json("dic_resources.json")

In [ ]:
credentials = credentials_dic[["copd"]]

In [ ]:
token <- TokenManager(credentials[["token_file"]])

In [ ]:
myconnection <- picsure::connect(url = credentials[["url"]],
                                 token = token)

In [ ]:
resource <- hpds::get.resource(myconnection,
                               resourceUUID = credentials[["resource"]])